In [11]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout
from keras_tqdm import TQDMNotebookCallback
import keras.callbacks

from sqlalchemy import create_engine
import json
import pickle

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [26]:
engine = create_engine('sqlite:///tab.db')
conn = engine.connect()
conn
dfo = pd.read_sql_table('race', conn)
print('{} races loaded!'.format(len(dfo)))

12813 races loaded!


In [27]:
category = 'H'
dfo = dfo.loc[dfo['race_type'].isin([category])]
print('{} {} races!'.format(len(dfo), category))

3364 H races!


In [28]:
dfo['results'] = dfo['results_data'].map(json.loads)
dfo['runners'] = dfo['runners_data'].map(json.loads)
dfo.head()

,id,meeting_name,location,venue_mnemonic,race_type,meeting_date,race_number,race_name,race_start_time,race_status,race_distance,results_data,num_runners,runners_data,results,runners
10,19,BATHURST,NSW,S,H,2017-09-13,1,LEXUS OF DUBBO PACE MS,2017-09-13 07:12:00,Paying,1730,"[[8], [9], [6], [7]]",NaN,"[{""runnerName"": ""NOTA BAD MOVE"", ""runnerNumber...","[[8], [9], [6], [7]]","[{'runnerName': 'NOTA BAD MOVE', 'runnerNumber..."
11,20,BATHURST,NSW,S,H,2017-09-13,2,DAVIS REMOVALS & STORAGE PACE MS,2017-09-13 07:39:00,Paying,1730,"[[5], [7], [3], [4]]",NaN,"[{""runnerName"": ""ASTA LEUGO"", ""runnerNumber"": ...","[[5], [7], [3], [4]]","[{'runnerName': 'ASTA LEUGO', 'runnerNumber': ..."
12,21,BATHURST,NSW,S,H,2017-09-13,3,MENANGLE COUNTRY SERIES HEAT MS,2017-09-13 08:14:00,Paying,1730,"[[1], [8], [2], [5]]",NaN,"[{""runnerName"": ""TIM MCGEE"", ""runnerNumber"": 5...","[[1], [8], [2], [5]]","[{'runnerName': 'TIM MCGEE', 'runnerNumber': 5..."
13,22,BATHURST,NSW,S,H,2017-09-13,4,BATHURST TOYOTA PACE MS,2017-09-13 08:40:00,Paying,1730,"[[3], [6], [8], [1]]",NaN,"[{""runnerName"": ""CHIEF FAIRMAN"", ""runnerNumber...","[[3], [6], [8], [1]]","[{'runnerName': 'CHIEF FAIRMAN', 'runnerNumber..."
14,23,BATHURST,NSW,S,H,2017-09-13,5,MENANGLE WHERE HORSES FLY HEAT MS,2017-09-13 09:10:00,Paying,1730,"[[8], [3], [6], [7]]",NaN,"[{""runnerName"": ""BILLY BRAVEHEART"", ""runnerNum...","[[8], [3], [6], [7]]","[{'runnerName': 'BILLY BRAVEHEART', 'runnerNum..."


In [29]:
# extract runners from races
data_all = pd.DataFrame()
for ri, race in dfo.iterrows():
#     print(race)

    num_runners = len([r for r in race['runners'] if r['odds_win']])

    # add num_runners to all runners
    for runner in race['runners']:
#         print(runner)
        runner['num_runners'] = num_runners
        runner['race_type'] = race['race_type']

    data_all = data_all.append(race['runners'])
    
data_all.tail(10)

,barrierNumber,claimAmount,finishingPosition,fixedOdds,num_runners,odds_perc,odds_scale,odds_win,parimutuel,race_type,rank_win,riderDriverName,runnerName,runnerNumber,trainerName
4,0,0,0,"{'returnWin': 9, 'returnWinOpen': 9, 'returnPl...",14,0.111111,0.087027,9.0,"{'bettingStatus': 'Normal', 'returnWin': 13.7,...",H,5,G LANNOO,VERTIGE DE VAUVERT,11,H GRIFT
5,0,0,0,"{'returnWin': 61, 'returnWinOpen': 61, 'return...",14,0.016393,0.012840,61.0,"{'bettingStatus': 'Normal', 'returnWin': 98.3,...",H,12,Y GERARD,ANAKIN SKYWALKER,12,V COLLARD
6,0,0,0,"{'returnWin': 26, 'returnWinOpen': 26, 'return...",14,0.038462,0.030125,26.0,"{'bettingStatus': 'Normal', 'returnWin': 47.1,...",H,10,TH E LONCKE,ULA DE NAVARY,13,TH LONCKE
7,0,0,4,"{'returnWin': 18, 'returnWinOpen': 18, 'return...",14,0.055556,0.043513,18.0,"{'bettingStatus': 'Normal', 'returnWin': 25.6,...",H,8,S OLIVIER,ATOUT DE MONTBRUN,2,PH DAUGEARD
8,0,0,0,"{'returnWin': 81, 'returnWinOpen': 81, 'return...",14,0.012346,0.009670,81.0,"{'bettingStatus': 'Normal', 'returnWin': 65.5,...",H,13,B BLACHET,TORBATO DE LUSSYL,3,B BLACHET
9,0,0,2,"{'returnWin': 4, 'returnWinOpen': 5, 'returnPl...",14,0.250000,0.195810,4.0,"{'bettingStatus': 'Normal', 'returnWin': 4.3, ...",H,2,F LECANU,BARAKA DE BOUGY,4,CH ECALARD
10,0,0,0,"{'returnWin': 21, 'returnWinOpen': 21, 'return...",14,0.047619,0.037297,21.0,"{'bettingStatus': 'Normal', 'returnWin': 40.6,...",H,9,TH VIET,ARLESIENNE,5,J F SENET
11,0,0,0,"{'returnWin': 15, 'returnWinOpen': 15, 'return...",14,0.066667,0.052216,15.0,"{'bettingStatus': 'Normal', 'returnWin': 14.8,...",H,6,P Y VERVA,BARON DU CHENE,6,F LEBLANC
12,0,0,0,"{'returnWin': 15, 'returnWinOpen': 15, 'return...",14,0.066667,0.052216,15.0,"{'bettingStatus': 'Normal', 'returnWin': 17.2,...",H,6,N BIAGINI,URIANO D'HAMELINE,7,A LE COURTOIS
13,0,0,0,"{'returnWin': 6.5, 'returnWinOpen': 6.5, 'retu...",14,0.153846,0.120499,6.5,"{'bettingStatus': 'Normal', 'returnWin': 6.9, ...",H,3,A GARANDEAU,BAR D'OR,1,A GARANDEAU


In [30]:
# drop scratched
data = data_all.dropna(subset=['odds_win'])
data = data[(data.odds_win > 0)]
data.describe()

,barrierNumber,claimAmount,finishingPosition,num_runners,odds_perc,odds_scale,odds_win,rank_win,runnerNumber
count,31860.000000,31860.0,31860.000000,31860.000000,31860.000000,31860.000000,31860.000000,31860.000000,31860.000000
mean,0.816008,0.0,1.009510,10.089203,0.132485,0.105587,22.654783,5.375455,5.729096
std,3.697640,0.0,1.400835,2.635573,0.148833,0.117419,25.987306,3.149860,3.377821
min,0.000000,0.0,0.000000,4.000000,0.001996,0.001677,1.010000,1.000000,1.000000
25%,0.000000,0.0,0.000000,8.000000,0.032258,0.026532,5.500000,3.000000,3.000000
50%,0.000000,0.0,0.000000,10.000000,0.076923,0.060370,13.000000,5.000000,5.000000
75%,0.000000,0.0,2.000000,12.000000,0.181818,0.140322,31.000000,7.000000,8.000000
max,70.000000,0.0,4.000000,20.000000,0.990099,0.834143,501.000000,19.000000,20.000000


In [31]:
# get label data
Y = data['finishingPosition'] == 1
Y = Y.astype(int)
Y.describe()

count    31860.000000
mean         0.105964
std          0.307796
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: finishingPosition, dtype: float64

In [32]:
# xp = fixed odds perc
xp = data['odds_perc']
xp.describe()

count    31860.000000
mean         0.132485
std          0.148833
min          0.001996
25%          0.032258
50%          0.076923
75%          0.181818
max          0.990099
Name: odds_perc, dtype: float64

In [33]:
# xs = odds scaled
xs = data['odds_scale']
xs.describe()

count    31860.000000
mean         0.105587
std          0.117419
min          0.001677
25%          0.026532
50%          0.060370
75%          0.140322
max          0.834143
Name: odds_scale, dtype: float64

In [34]:
# xr = win rank
xr = data['rank_win']
xr.describe()

count    31860.000000
mean         5.375455
std          3.149860
min          1.000000
25%          3.000000
50%          5.000000
75%          7.000000
max         19.000000
Name: rank_win, dtype: float64

In [35]:
# xn = get num runners
xn = data['num_runners']
xn.describe()

count    31860.000000
mean        10.089203
std          2.635573
min          4.000000
25%          8.000000
50%         10.000000
75%         12.000000
max         20.000000
Name: num_runners, dtype: float64

In [36]:
X = pd.concat([xp, xs, xr, xn], axis=1)
X.describe()
# sns.pairplot(pd.concat([X, Y], axis=1))

,odds_perc,odds_scale,rank_win,num_runners
count,31860.000000,31860.000000,31860.000000,31860.000000
mean,0.132485,0.105587,5.375455,10.089203
std,0.148833,0.117419,3.149860,2.635573
min,0.001996,0.001677,1.000000,4.000000
25%,0.032258,0.026532,3.000000,8.000000
50%,0.076923,0.060370,5.000000,10.000000
75%,0.181818,0.140322,7.000000,12.000000
max,0.990099,0.834143,19.000000,20.000000


In [37]:
# For a single-input model with 2 classes (binary classification):

n = len(X.columns)
print('input dimension = {}'.format(n))

epochs = 1000
print('epochs = {}'.format(epochs))

tag = '{}30x30'.format(category)
file_name = '/Users/jaco/code/tabby/models/{}.h5'.format(tag)

try:
    model = load_model(file_name)
except OSError:
    model = Sequential()
    model.add(Dense(30, activation='relu', input_dim=n))
    model.add(Dense(30, activation='relu'))
#     model.add(Dense(40, activation='relu'))
#     model.add(Dense(40, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

input dimension = 4
epochs = 1000


In [38]:
# Train the model, iterating on the data in batches of 32 samples
tqdm = TQDMNotebookCallback()

tbCallBack = keras.callbacks.TensorBoard(
    log_dir='/Users/jaco/code/tabby/summary/{}'.format(tag), 
    histogram_freq=0,
    write_graph=True,
    write_images=True)

model.fit(
    X.as_matrix(), 
    Y.as_matrix(),
    validation_split=0.2,
    shuffle=True,
    epochs=epochs,
    batch_size=32,
    verbose=0,
    callbacks=[tqdm, tbCallBack])

# creates a HDF5 file 'my_model.h5'
model.save(file_name)